In [ ]:
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
np.set_printoptions(linewidth=150)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from src.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from src.data import mdl

from src.symbols import *
from src.parameters import *
import src.symbols as symbols
from src import prime_system
from src.models import regression
from src.visualization.regression import show_pred
from src.visualization.plot import track_plot

## Load models:
# (Uncomment these for faster loading):
import src.models.vmm_nonlinear_EOM  as vmm 
from src.models.vmm import ModelSimulator
from docs.book.example_1 import ship_parameters, df_parameters


if os.name == 'nt':
    plt.style.use('../docs/book/book.mplstyle')  # Windows
    
from src.visualization.plot import track_plot, plot

In [ ]:
from src.extended_kalman_vmm import ExtendedKalman

## Load test

In [ ]:
id=22773
df, units, meta_data = mdl.load(dir_path = '../data/raw', id=id)
df.index = df.index.total_seconds()
df.index-=df.index[0]
df['x0']-=df.iloc[0]['x0']
df['y0']-=df.iloc[0]['y0']
df['psi']-=df.iloc[0]['psi']


In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(10,10)
track_plot(df=df, lpp=meta_data.lpp, x_dataset='x0', y_dataset='y0',  psi_dataset='psi', beam=meta_data.beam, ax=ax);

## Filtering

In [ ]:
parameters = df_parameters["prime"].copy()
ek = ExtendedKalman(vmm=vmm, 
                    parameters=parameters, 
                    ship_parameters=ship_parameters)


In [ ]:
from numpy import cos as cos
from numpy import sin as sin
from src.data.lowpass_filter import lowpass_filter

df_lowpass = df.copy()
t = df_lowpass.index
ts = np.mean(np.diff(t))
fs = 1/ts

position_keys = ['x0','y0','psi']
for key in position_keys:
    df_lowpass[key] = lowpass_filter(data=df_lowpass[key], fs=fs, cutoff=1, order=1)

df_lowpass['x01d_gradient'] = x1d_ = np.gradient(df_lowpass['x0'], t)
df_lowpass['y01d_gradient'] = y1d_ = np.gradient(df_lowpass['y0'], t)
df_lowpass['r'] = r_ = np.gradient(df_lowpass['psi'], t)

psi_ = df_lowpass['psi']

df_lowpass['u'] = x1d_*cos(psi_) + y1d_*sin(psi_)
df_lowpass['v'] = -x1d_*sin(psi_) + y1d_*cos(psi_)

velocity_keys = ['u','v','r']
for key in velocity_keys:
    df_lowpass[key] = lowpass_filter(data=df_lowpass[key], fs=fs, cutoff=1, order=1)

In [ ]:
data = df.copy()
data['u'] = df_lowpass['u']
data['v'] = df_lowpass['v']
data['r'] = df_lowpass['r']
data=data.iloc[200:-100]
data.index-=data.index[0]

P_prd = np.diag([0.1, 0.1, np.deg2rad(0.01), 0.001, 0.001, np.deg2rad(0.001)])

Qd = np.diag([0.01, 0.01, np.deg2rad(0.1)])  #process variances: u,v,r

Cd = np.array([
    [1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0],
])

E = np.array([
    [0,0,0],
    [0,0,0],
    [0,0,0],
    [1,0,0],
    [0,1,0],
    [0,0,1],
    ],
)


ys = data[['x0','y0','psi']].values
h_m = h_ = np.mean(np.diff(data.index))

x0 = np.concatenate((
    data.iloc[0][['x0','y0','psi']].values,
    data.iloc[0][['u','v','r']].values))

us = data['delta'].values

error_max_pos = 0.05
sigma_pos = error_max_pos/3
variance_pos = sigma_pos**2

error_max_psi = np.deg2rad(0.5)
sigma_psi = error_max_psi/3
variance_psi = sigma_psi**2

Rd = np.diag([variance_pos, variance_pos, variance_psi])

h = np.mean(np.diff(data.index))

time_steps = ek.filter(
        x0=x0, P_prd=P_prd, h=h, us=us, ys=ys, Qd=Qd, Rd=Rd, E=E, Cd=Cd
    )



x_hats = np.array([time_step["x_hat"].flatten() for time_step in time_steps]).T
time = np.array([time_step["time"] for time_step in time_steps]).T
Ks = np.array([time_step["K"] for time_step in time_steps]).T
variances = np.array([np.diagonal(time_step["P_hat"]) for time_step in time_steps]).T
stds = np.sqrt(variances)

In [ ]:
df_kalman = pd.DataFrame(data=x_hats.T, index=time, columns=['x0','y0','psi','u','v','r'])
df_kalman['delta'] = us

for key in ['u','v','r']:
    df_kalman[f'{key}1d'] = np.gradient(df_kalman[key], df_kalman.index)

In [ ]:
smooth_time_steps = ek.smoother()

## Post process rts smoother:
x_hats = np.array(
    [time_step["x_hat"].flatten() for time_step in smooth_time_steps]
).T
time = np.array([time_step["time"] for time_step in smooth_time_steps]).T
df_rts = pd.DataFrame(data=x_hats.T, index=time, columns=['x0','y0','psi','u','v','r'])
df_rts["delta"] = us

for key in ['u','v','r']:
    df_rts[f'{key}1d'] = np.gradient(df_rts[key], df_kalman.index)

In [ ]:
dataframes = {
    'Mesurement' : data,
    'Kalman filter' : df_kalman,
    'RTS': df_rts,
}

fig,ax=plt.subplots()
styles = {
    'Mesurement' : {
        'linestyle' : '',
        'marker' : '.',
        'ms' : 1,
        'zorder':-10,
    },
    
    'Kalman filter' : {
        'lw' : 2,
    },
    
    
}

for label,df_ in dataframes.items():
    track_plot(
            df=df_,
            lpp=ship_parameters["L"],
            beam=ship_parameters["B"],
            ax=ax,
            label=label,
            plot_boats=False,
            **styles.get(label,{})
        );
ax.legend()


plot(dataframes = dataframes, 
     fig_size=(10,15), 
     styles = ['r-','g-','b-'],
     keys=['x0','y0','psi','u','v','r','u1d','v1d','r1d']);

In [ ]:
ek.parameters

In [ ]:
ek2 = ek.copy()